In [1]:
%matplotlib notebook
import numpy as np                  # This is the standard way of importing numpy
import matplotlib.pyplot as plt     # and this is the usual way to load matplotlib
import pandas as pd                 # Pandas is convenient for tabular data

In [3]:
print("hellowordsorthings")

hellowordsorthings


In [4]:
print("hello, whats up") # ask the user what is above them, nicely
x = 0
for i in range(100):
    x += 1
print(x)

hello, whats up
100


In [5]:
print("Hello")
for i in range(6):
    print(i)

Hello
0
1
2
3
4
5


In [6]:
print("no")

no
